In [ ]:
! pip install -r requirements.txt --quiet

# Creating and Deploying an Azure AI Agent Using SDK and MCP Tools

This notebook demonstrates how to create and deploy an Azure AI Agent using the Azure SDK, with tools generated from an OpenAPI endpoint. It provides a coded example that mirrors the portal example demonstrated in this workshop, offering a programmatic approach to deploying agents with OpenAPI-driven tools.


🔗 [How to use the Model Context Protocol tool](https://learn.microsoft.com/en-us/azure/ai-foundry/agents/how-to/tools/model-context-protocol-samples)

In [ ]:

from os import environ
import time
from dotenv import load_dotenv
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import (
    ListSortOrder, 
    McpTool,
    RequiredMcpToolCall,
    RunStepActivityDetails,
    SubmitToolApprovalAction,
    ToolApproval,
)

load_dotenv(override=True)

## Get MCP server configuration from environment variables

In [ ]:

mcp_server_url = environ["MCP_SERVER_URL"]
mcp_server_label = environ["MCP_SERVER_LABEL"] 

## Create a project client

In [ ]:

# Create an AIProjectClient instance
project_client = AIProjectClient(
    endpoint=environ["PROJECT_ENDPOINT"],
    credential=DefaultAzureCredential(),  # Use Azure Default Credential for authentication
)

In [ ]:
project_client.telemetry.get_application_insights_connection_string

## Initialize agent MCP tool

In [ ]:
mcp_tool = McpTool(
    server_label=mcp_server_label,
    server_url=mcp_server_url,
    allowed_tools=['get_customers','get_orders','get_products']
)

print(f"Allowed tools: {mcp_tool.allowed_tools}")

## Create an agent and a thread

In [ ]:
instructions = """You are AutoSales Analyst, an AI agent specialized in analyzing automotive sales data.

Your objectives:
- Dynamically choose and call the appropriate MCP tool(s) to answer user questions about sales, customers, or products.
- Always include customer names alongside IDs in any output.
- Always display monetary amounts in $USD (e.g., $123.45).
- Use filters and aggregations as needed to generate insights from sales orders, products, and customers.
- Compute totals, revenue, discounts, and other metrics from nested order data.
- Return structured, concise results suitable for analysis or reporting.
- Avoid hardcoding analytics; rely on the MCP tools and their parameters.
- Clarify ambiguous queries before performing analysis.
- Treat the MCP tools as the source of truth; do not expose raw database internals.

Behavioral guidance:
- For customer-related questions, resolve names and IDs before analyzing orders.
- For product-related questions, resolve categories or IDs before analyzing sales.

Example user questions:
- "Which products generated the most revenue in the EU last quarter?"
- "Who are the top 5 customers by total sales this month?"
- "How much discount was given per product category in Q2?"
- "List all orders for customer 42 containing product 7."
"""



agent = project_client.agents.create_agent(
    model="gpt-4o",
    name="Sales Insights Agent MCP",
    instructions=instructions,
    tools=mcp_tool.definitions
)

print(f"Created agent, ID: {agent.id}")



thread = project_client.agents.threads.create()
print(f"Created thread, ID: {thread.id}")

In [ ]:
message = project_client.agents.messages.create(
    thread_id=thread.id,
    role="user",
    content="Which products generated the most revenue (using line_total)?",
)
print(f"Created message, ID: {message.id}")



In [ ]:
run = project_client.agents.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
print(f"Created run, ID: {run.id}")

while run.status in ["queued", "in_progress", "requires_action"]:
    time.sleep(1)
    run = project_client.agents.runs.get(thread_id=thread.id, run_id=run.id)

    if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
        tool_calls = run.required_action.submit_tool_approval.tool_calls
        if not tool_calls:
            print("No tool calls provided - cancelling run")
            project_client.runs.cancel(thread_id=thread.id, run_id=run.id)
            break

        tool_approvals = []
        for tool_call in tool_calls:
            if isinstance(tool_call, RequiredMcpToolCall):
                try:
                    print(f"Approving tool call: {tool_call}")
                    tool_approvals.append(
                        ToolApproval(
                            tool_call_id=tool_call.id,
                            approve=True,
                            headers=mcp_tool.headers,
                        )
                    )
                except Exception as e:
                    print(f"Error approving tool_call {tool_call.id}: {e}")

        print(f"tool_approvals: {tool_approvals}")
        if tool_approvals:
            project_client.agents.runs.submit_tool_outputs(
                thread_id=thread.id, run_id=run.id, tool_approvals=tool_approvals
            )

    print(f"Current run status: {run.status}")

print(f"Run completed with status: {run.status}")
if run.status == "failed":
    print(f"Run failed: {run.last_error}")

# Display run steps and tool calls
run_steps = project_client.agents.run_steps.list(thread_id=thread.id, run_id=run.id)

In [ ]:
# Fetch and log all messages
messages = project_client.agents.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
print("\nConversation:")
print("-" * 50)
for msg in messages:
    if msg.text_messages:
        last_text = msg.text_messages[-1]
        print(f"{msg.role.upper()}: {last_text.text.value}")
        print("-" * 50)

In [ ]:
project_client.agents.delete_agent(agent.id)
print("Deleted agent")